In [5]:
from pyspark.sql import SparkSession
import numpy as np

In [6]:
spark = SparkSession.builder.appName("linkage").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
rawblocks = sc.textFile("data")

In [9]:
rawblocks.first()

u'"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"'

In [10]:
rawblocks.count()

5749142

# Bringing Data from the cluster to the Client


In [11]:
head = rawblocks.take(10)

In [12]:
head

[u'"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"',
 u'37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE',
 u'39086,47614,1,?,1,?,1,1,1,1,1,TRUE',
 u'70031,70237,1,?,1,?,1,1,1,1,1,TRUE',
 u'84795,97439,1,?,1,?,1,1,1,1,1,TRUE',
 u'36950,42116,1,?,1,1,1,1,1,1,1,TRUE',
 u'42413,48491,1,?,1,?,1,1,1,1,1,TRUE',
 u'25965,64753,1,?,1,?,1,1,1,1,1,TRUE',
 u'49451,90407,1,?,1,?,1,1,1,1,0,TRUE',
 u'39932,40902,1,?,1,?,1,1,1,1,1,TRUE']

In [13]:
header = rawblocks.first()
header

u'"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"'

# Shipping code from the client to the cluster

In [15]:
def isheader(x):
    if 'id_1' not in x:
        return x

In [16]:
noheader = rawblocks.filter(isheader)

In [17]:
noheader.first()

u'37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE'

# From RDDs to Data Frames

In [19]:
prev = spark.read.csv("data")

In [20]:
prev.show()

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|  

In [21]:
parsed = spark.read.option("header", "true")\
                   .option("nullValue", "?")\
                   .option("inferSchema", "true")\
                   .csv("data")                                                    

In [22]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



# Analyzing Data with the DataFrame API

In [23]:
parsed.count()

5749132

In [24]:
parsed.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

In [32]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [33]:
parsed.\
    groupBy("is_match").\
    count().\
    orderBy(desc("count")).\
    show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [34]:
parsed.agg(avg("cmp_sex"), stddev("cmp_sex")).show()

+-----------------+--------------------+
|     avg(cmp_sex)|stddev_samp(cmp_sex)|
+-----------------+--------------------+
|0.955001381078048| 0.20730111116897781|
+-----------------+--------------------+



In [35]:
parsed.createOrReplaceTempView("linkage")

In [36]:
spark.sql("""
          SELECT is_match, COUNT(*) cnt
          FROM linkage
          GROUP BY is_match
          ORDER BY cnt DESC
          """).show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



# Fast Summary Statistics for DataFrames

In [38]:
summary = parsed.describe()
summary.show()

+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|       cmp_fname_c1|      cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|            5748125|            103698|           5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935| 0.7129024704437266|0.9000176718903189|0.315627819308

In [39]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+-------------------+------------------+
|summary|       cmp_fname_c1|      cmp_fname_c2|
+-------+-------------------+------------------+
|  count|            5748125|            103698|
|   mean| 0.7129024704437266|0.9000176718903189|
| stddev|0.38875835961628014|0.2713176105782334|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [41]:
matches = parsed.where("is_match = true")

In [42]:
matchSummary = matches.describe()

In [50]:
misses = parsed.where("is_match= false")

In [51]:
missSummary = misses.describe()

# Pivoting and Reshaping DataFrames

In [55]:
summary.printSchema()

root
 |-- summary: string (nullable = true)
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: string (nullable = true)
 |-- cmp_plz: string (nullable = true)



In [57]:
schema = summary.schema

In [80]:
import pandas as pd

In [149]:
def transform(spark_df):
    """
    Transpose the spark dataframe and change the column datatype"""
    df = spark_df.toPandas()
    df = df.transpose()
    new_header = df.iloc[0]
    df = df[1:]
    df = df.rename(columns = new_header)
    df['field'] = df.index
    df[['count', 'mean', 'stddev', 'min', 'max']] = df[['count', 'mean','stddev', 'min', 'max']].apply(pd.to_numeric)
    spark_df = spark.createDataFrame(df)
    spark_df.printSchema()
    return spark_df
    

In [150]:
summary_df = transform(summary)

root
 |-- count: long (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)
 |-- field: string (nullable = true)



In [152]:
matchSummary = transform(matchSummary)

root
 |-- count: long (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)
 |-- field: string (nullable = true)



In [153]:
missSummary = transform(missSummary)

root
 |-- count: long (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)
 |-- field: string (nullable = true)



# Joining DataFrames and Selecting Features

In [154]:
matchSummary.createOrReplaceTempView("match_desc")
missSummary.createOrReplaceTempView("miss_desc")

In [155]:
spark.sql("""
      SELECT a.field, a.count + b.count total, a.mean - b.mean delta
      FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
      WHERE a.field NOT IN ("id_1", "id_2")
      ORDER BY delta DESC, total DESC
      """).show()

+------------+-------+-------------------+
|       field|  total|              delta|
+------------+-------+-------------------+
|     cmp_plz|5736289| 0.9563812499852177|
|cmp_lname_c2|   2464| 0.8064147192926266|
|      cmp_by|5748337| 0.7762059675300511|
|      cmp_bd|5748337| 0.7754423117834038|
|cmp_lname_c1|5749132| 0.6838772482594512|
|      cmp_bm|5748337| 0.5109496938298687|
|cmp_fname_c1|5748125| 0.2854529057459947|
|cmp_fname_c2| 103698|0.09104268062280174|
|     cmp_sex|5749132|0.03240818525033273|
+------------+-------+-------------------+



A good feature has two properties: it tends to have significantly different values for matches and nonmatches (so the difference between the means will be large) and it occurs often enough in the data that we can rely on it to be regularly available for any pair of records. By this measure, cmp_fname_c2 isn’t very useful because it’s missing a lot of the time and the difference in the mean value for matches and nonmatches is relatively small—0.09, for a score that ranges from 0 to 1. The cmp_sex feature also isn’t particularly helpful because even though it’s available for any pair of records, the difference in means is just 0.03.


Features cmp_plz and cmp_by, on the other hand, are excellent. Theyalmost always occur for any pair of records, and there is a very large difference in the mean values (more than 0.77 for both features.) Features cmp_bd, cmp_lname_c1, and cmp_bm also seem beneficial: they are generally available in the data set and the difference, in mean values for matches and nonmatches are substantial.


Features cmp_fname_c1 and cmp_lname_c2 are more of a mixed bag: cmp_fname_c1 doesn’t discriminate all that well (the difference in the means is only 0.28) even though it’s usually available for a pair of records, whereas cmp_lname_c2 has a large difference in the means but it’s almost always missing. It’s not quite obvious under what circumstances we should include these features in our model based on this data.For now, we’re going to use a simple scoring model that ranks the similarity of pairs of records based on the sums of the values of the obviously good features: cmp_plz, cmp_by, cmp_bd, cmp_lname_c1, and cmp_bm. For the few records where the values of these features are missing, we’ll use 0 in place of the null value in our sum. We can get a rough feel for the performance of our simple model by creating a data frame of the computed scores and the value of the is_match column and evaluating how well the score discriminates between matches and nonmatches at various thresholds.